In [1]:
%load_ext autoreload
%autoreload 2

from nn import EncoderRNN, DecoderRNN, Seq2Seq
import utils
import cPickle as pickle
import torch
from tqdm import tqdm
import random
# import tensorflow as tf

In [2]:
# lang, pairs = utils.prepare_data('../data/cmd/dialogs.txt')
lang, pairs = utils.prepare_data('../res/friends.txt', persona=True)
# lang, pairs = pickle.load(open('data.pkl'))

Reading lines...
Done


In [ ]:
pickle.dump((lang, pairs), open('data.pkl', 'wb'))

# Testing if everything works

In [3]:
hidden_size = 10
enc_size = hidden_size
dec_size = hidden_size
learning_rate = 0.01
emb_dims = 5
max_length = 10
lang.max_length = max_length
seq2seq = Seq2Seq(lang, enc_size, dec_size, emb_dims, max_length, learning_rate, 
                  persona_size=20, attention=True, persona=True)

AttributeError: type object 'object' has no attribute '__getattr__'

In [5]:
from tqdm import tqdm
test_pair = [('Joey:how are you', 'Chandler:pretty good'), ('Chandler:what are you doing', 'Joey:nothing')]
for i in tqdm(xrange(500)):
    _, loss = seq2seq.forward(random.choice(test_pair))

  0%|          | 0/500 [00:00<?, ?it/s]


RuntimeError: size mismatch at /b/wheel/pytorch-src/torch/lib/THC/generic/THCTensorMathBlas.cu:243

In [11]:
output, _ = seq2seq.forward(('Rachel:how are you', 'Ross: '), False)
print output

Variable containing:
  31
 205
  13
   1
[torch.cuda.LongTensor of size 4x1 (GPU 0)]

pretty


In [ ]:
lst = [34, 128, 12, 35, 672, 2333, 1]
for i in lst:
    print seq2seq.lang.index2word[i],
print

# Training the model

In [ ]:
hidden_size = 512
enc_size = hidden_size
dec_size = hidden_size
learning_rate = 1e-5
emb_dims = 200
max_length = 30
lang.max_length = max_length
seq2seq = Seq2Seq(lang, enc_size, dec_size, emb_dims, max_length, learning_rate, reload_model=False,
                 attention=True, )

In [ ]:
import random

for i in tqdm(xrange(1000)):
    # Choose a random pair and train with that
    r_pair = random.choice(pairs)
    out, loss = seq2seq.forward(r_pair)
    del loss
    del out

In [ ]:
torch.save(seq2seq.encoder, '../models/encoder.pth')
torch.save(seq2seq.decoder, '../models/decoder.pth')

In [ ]:
output, _ = seq2seq.forward(('who are you', ''), False)

In [ ]:
output